# Description

TODO

# Environment variables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import conf

In [3]:
N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

2

In [4]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=2
env: OPEN_BLAS_NUM_THREADS=2
env: NUMEXPR_NUM_THREADS=2
env: OMP_NUM_THREADS=2


# Modules

In [5]:
import warnings
from pathlib import Path

# import statsmodels.api as sm
# from statsmodels.stats.multitest import multipletests
import numpy as np
import pandas as pd

# from sklearn.preprocessing import scale
# from tqdm import tqdm

# from gls import GLSPhenoplier

# Settings

In [6]:
OUTPUT_DIR = conf.RESULTS["GLS"]
display(OUTPUT_DIR)

assert OUTPUT_DIR.exists()

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases/phenoplier_asthma/results/gls')

# Get results files

In [9]:
INPUT_FILES_PER_COHORT = {
    "phenomexcan": list(OUTPUT_DIR.glob("gls_phenotypes*combined*phenomexcan*.pkl")),
    "emerge": list(OUTPUT_DIR.glob("gls_phenotypes*combined*emerge*.pkl")),
}

In [10]:
display(INPUT_FILES_PER_COHORT)

{'phenomexcan': [PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases/phenoplier_asthma/results/gls/gls_phenotypes-combined-phenomexcan.pkl')],
 'emerge': [PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases/phenoplier_asthma/results/gls/gls_phenotypes-combined-emerge.pkl')]}

# PhenomeXcan

In [12]:
phenomexcan_df = pd.read_pickle(INPUT_FILES_PER_COHORT["phenomexcan"][0]).drop(
    columns=["pvalue_twosided", "part_k", "cluster_id"]
)

In [13]:
phenomexcan_df.shape

(100, 5)

In [14]:
phenomexcan_df = phenomexcan_df.replace(
    {
        "phenotype": {
            "6152_100-Blood_clot_DVT_bronchitis_emphysema_asthma_rhinitis_eczema_allergy_diagnosed_by_doctor_None_of_the_above": "Asthma/allergic rhinitis/atopic dermatitis",
            "6152_8-Blood_clot_DVT_bronchitis_emphysema_asthma_rhinitis_eczema_allergy_diagnosed_by_doctor_Asthma": "Doctor diagnosed asthma",
            "6152_9-Blood_clot_DVT_bronchitis_emphysema_asthma_rhinitis_eczema_allergy_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema": "Doctor diagnosed allergic rhinitis or atopic dermatitis",
        }
    }
)

In [20]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    tmp = phenomexcan_df.sort_values("fdr").head(20)
    display(tmp)

,phenotype,lv,coef,pvalue,fdr
36,Asthma/allergic rhinitis/atopic dermatitis,LV948,0.086323,3.299886e-12,2.753432e-10
34,Asthma/allergic rhinitis/atopic dermatitis,LV96,0.085516,5.506865e-12,2.753432e-10
80,asthma,LV207,0.084608,1.191771e-11,3.972569e-10
49,Doctor diagnosed asthma,LV207,0.083574,2.052668e-11,5.073843e-10
73,Doctor diagnosed allergic rhinitis or atopic dermatitis,LV948,0.082493,2.536922e-11,5.073843e-10
94,asthma,LV948,0.076148,1.011494e-09,1.685824e-08
92,asthma,LV96,0.074659,2.171634e-09,3.102334e-08
30,Asthma/allergic rhinitis/atopic dermatitis,LV942,0.072792,3.880992e-09,4.851240e-08
77,Doctor diagnosed allergic rhinitis or atopic dermatitis,LV96,0.072162,4.840323e-09,5.378137e-08
53,Doctor diagnosed asthma,LV948,0.071636,8.440642e-09,8.440642e-08


# eMERGE

## Trait info

In [25]:
input_filepath = conf.EMERGE["DESC_FILE_WITH_SAMPLE_SIZE"]
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases/phenoplier/data/emerge/eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt')

In [38]:
emerge_traits_df = pd.read_csv(
    input_filepath,
    sep="\t",
    dtype={"phecode": str},
    usecols=["phecode", "phenotype", "category"],
)

In [39]:
emerge_traits_df = emerge_traits_df.rename(
    columns={
        "phenotype": "phecode_phenotype",
        "category": "phecode_category",
    }
)

In [43]:
assert emerge_traits_df["phecode"].is_unique
emerge_traits_df = emerge_traits_df.set_index("phecode")

In [44]:
emerge_traits_df.shape

(309, 2)

In [45]:
emerge_traits_df.head()

,phecode_phenotype,phecode_category
phecode,,
008,Intestinal infection,infectious diseases
008.5,Bacterial enteritis,infectious diseases
008.52,Intestinal infection due to C. difficile,infectious diseases
038,Septicemia,infectious diseases
038.3,Bacteremia,infectious diseases


In [46]:
emerge_code_desc_map = emerge_traits_df["phecode_phenotype"].to_dict()

In [66]:
emerge_code_category_map = emerge_traits_df["phecode_category"].to_dict()

## Association results

In [71]:
emerge_df = pd.read_pickle(INPUT_FILES_PER_COHORT["emerge"][0]).drop(
    columns=["pvalue_twosided"]
)

In [72]:
emerge_df.insert(
    1, "phenotype_desc", emerge_df["phenotype"].apply(lambda x: emerge_code_desc_map[x])
)
emerge_df.insert(
    2,
    "phenotype_category",
    emerge_df["phenotype"].apply(lambda x: emerge_code_category_map[x]),
)

In [73]:
emerge_df.shape

(6180, 7)

In [74]:
emerge_df.head()

,phenotype,phenotype_desc,phenotype_category,lv,coef,pvalue,fdr
0,008,Intestinal infection,infectious diseases,LV948,-0.010377,0.791969,0.998269
1,008,Intestinal infection,infectious diseases,LV96,-0.011360,0.813072,0.998269
2,008,Intestinal infection,infectious diseases,LV335,0.004037,0.375220,0.998269
3,008,Intestinal infection,infectious diseases,LV679,-0.016359,0.906661,0.998269
4,008,Intestinal infection,infectious diseases,LV207,0.010934,0.195141,0.975890


In [75]:
# phenomexcan_df = phenomexcan_df.replace(
#     {
#         "phenotype": {
#             "6152_100-Blood_clot_DVT_bronchitis_emphysema_asthma_rhinitis_eczema_allergy_diagnosed_by_doctor_None_of_the_above": "Asthma/allergic rhinitis/atopic dermatitis",
#             "6152_8-Blood_clot_DVT_bronchitis_emphysema_asthma_rhinitis_eczema_allergy_diagnosed_by_doctor_Asthma": "Doctor diagnosed asthma",
#             "6152_9-Blood_clot_DVT_bronchitis_emphysema_asthma_rhinitis_eczema_allergy_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema": "Doctor diagnosed allergic rhinitis or atopic dermatitis",
#         }
#     }
# )

In [76]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    tmp = emerge_df[emerge_df.fdr < 0.10]
    tmp = tmp.sort_values("fdr").head(20)
    display(tmp)

,phenotype,phenotype_desc,phenotype_category,lv,coef,pvalue,fdr
4225,509,"Respiratory failure, insufficiency, arrest",respiratory,LV207,0.059579,0.000001,0.008138
3371,429,Ill-defined descriptions and complications of heart disease,circulatory system,LV444,0.051574,0.000015,0.044697
5212,591,Urinary tract infection,genitourinary,LV564,0.051078,0.000022,0.044697
3319,428.2,Heart failure NOS,circulatory system,LV390,0.047641,0.000077,0.059729
3716,443.9,"Peripheral vascular disease, unspecified",circulatory system,LV644,0.047482,0.000066,0.059729
4031,480,Pneumonia,respiratory,LV882,0.045475,0.000069,0.059729
4072,483,Acute bronchitis and bronchiolitis,respiratory,LV882,0.046222,0.000059,0.059729
4681,564,Functional digestive disorders,digestive,LV882,0.046387,0.000051,0.059729
589,244,Hypothyroidism,endocrine/metabolic,LV948,0.047649,0.000113,0.065554
3076,427.11,Paroxysmal supraventricular tachycardia,circulatory system,LV948,0.046780,0.000113,0.065554
